## Build model

In [ ]:
%%bash

if [ ! -d ./data ]; then
    mkdir ./data
fi

curl https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz --output data/cooking.stackexchange.tar.gz && tar xvzf data/cooking.stackexchange.tar.gz -C data
head -n 12404 data/cooking.stackexchange.txt > data/cooking.train
tail -n 3000 data/cooking.stackexchange.txt > data/cooking.valid

In [ ]:
import fasttext
import numpy as np

In [ ]:
model = fasttext.train_supervised(input="./data/cooking.train", lr=0.8, epoch=70, loss='hs')

In [ ]:
model.test("./data/cooking.valid")

In [ ]:
class_names = model.labels

In [ ]:
class_names = [s.replace("__label__", "") for s in class_names]

In [ ]:
k = len(class_names)
idx_to_labels = {i:k for k, i in zip(class_names, range(k))}
labels_to_idx = {k:i for k, i in zip(class_names, range(k))}

In [ ]:
labels_to_idx['cake'], k

In [ ]:
def get_predictions(model, text_list):
    
    predictions = model.predict(text_list, k=k)
    x, y = predictions
    
    probabilities_full_list = []
    for label_list, prob_list in zip(x, y):
        label_prob_pair_dict = {}
        for lbl, prob in zip(label_list, prob_list):
            label_prob_pair_dict[lbl.replace("__label__", "")] = prob
        probabilities_list = []
        for cls in class_names:
            if cls in label_prob_pair_dict:
                p = label_prob_pair_dict[cls]
                probabilities_list.append(p)
            else:
                probabilities_list.append(0.0)
        probabilities_full_list.append(probabilities_list)
        
    return np.array(probabilities_full_list)

In [ ]:
%%time
results = get_predictions(model, ["cake"]*1000*10)

# Unbox

In [ ]:
import unboxapi
from unboxapi.models import ModelType
from unboxapi.tasks import TaskType

client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

### Upload model

In [ ]:
# Comment this out and uncomment the next section to load the project
project = client.create_project(
    name="Fasttext Demo",
    description="Fasttext Demo Project",
    task_type=TaskType.TextClassification,
)

# Use this for loading the project on subsequent runs
'''
project = client.load_project(
    name="Fasttext Demo"
)
'''

ml_model = project.add_model(
    function=get_predictions, 
    model=model,
    model_type=ModelType.fasttext,
    class_names=class_names,
    name='Cooking Fast Text',
    description='this is my fasttext model',
)
ml_model.to_dict()